In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
data = pd.read_csv('combined_data.csv')
le = LabelEncoder()
label=le.fit_transform(data['Protocol'])
data['Protocol']=label

In [ ]:
data = data.drop(data.columns[0], axis=1)

In [ ]:
data.describe()

,Time,Protocol,Length,Target
count,3.172390e+05,317239.000000,317239.000000,317239.000000
mean,6.745880e+01,14.254222,492.194393,0.945650
std,1.916876e+02,3.108322,952.918346,0.226708
min,3.610000e-07,0.000000,42.000000,0.000000
25%,5.406614e-01,14.000000,66.000000,1.000000
50%,1.879724e+00,14.000000,74.000000,1.000000
75%,5.238090e+01,17.000000,1152.000000,1.000000
max,9.699905e+02,17.000000,44954.000000,1.000000


In [ ]:
data=data.dropna()

In [ ]:
corr_matrix = data.corr()
corr_matrix

,Time,Protocol,Length,Target
Time,1.000000,-0.167834,-0.047586,-0.956120
Protocol,-0.167834,1.000000,-0.156231,0.104349
Length,-0.047586,-0.156231,1.000000,0.016479
Target,-0.956120,0.104349,0.016479,1.000000


In [ ]:
X = data[['Time','Protocol','Length']]
y = data['Target']

In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)
importances = rf.feature_importances_
indices = sorted(range(len(importances)), key=lambda i: importances[i], reverse=True)

In [ ]:
X=data[['Time','Length']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-0.33721991, -0.45984536],
       [-0.33719915, -0.45984536],
       [-0.33694828, -0.45984536],
       ...,
       [-0.33453992,  1.07229252],
       [-0.3345398 ,  1.07229252],
       [-0.33453965,  1.07229252]])

In [ ]:
X = data.iloc[:,:-1].values
X[:,0]=X_scaled[:,0]*-1
X

array([[0.3372199095369218, '192.168.112.128', '49.44.194.34', 14, 54,
        '40492  >  80 [ACK] Seq=1 Ack=1 Win=64008 Len=0'],
       [0.3371991473364919, '192.168.112.128', '49.44.194.34', 14, 54,
        '40510  >  80 [ACK] Seq=1 Ack=1 Win=63936 Len=0'],
       [0.33694828393219456, '192.168.112.128', '152.195.38.76', 14, 54,
        '49232  >  80 [ACK] Seq=1 Ack=1 Win=63812 Len=0'],
       ...,
       [0.3345399221130892, '192.168.112.128', '192.168.112.129', 14,
        1514,
        '49876  >  80 [ACK] Seq=12450561 Ack=1 Win=501 Len=1448 TSval=1204965619 TSecr=777797 [TCP segment of a reassembled PDU]'],
       [0.3345397970135399, '192.168.112.128', '192.168.112.129', 14,
        1514,
        '52492  >  80 [ACK] Seq=10219313 Ack=1 Win=501 Len=1448 TSval=1204965619 TSecr=777797 [TCP segment of a reassembled PDU]'],
       [0.3345396510049417, '192.168.112.128', '192.168.112.129', 14,
        1514,
        '52456  >  80 [PSH, ACK] Seq=10889281 Ack=1 Win=501 Len=1448 TSval=12049

In [ ]:
y

0         0
1         0
2         0
3         0
4         0
         ..
317234    1
317235    1
317236    1
317237    1
317238    1
Name: Target, Length: 317239, dtype: int64

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(X,y,random_state=42,train_size=0.8)
X_train = np.asarray(X_train)
X_test= np.asarray(X_test)
y_train=np.asarray(y_train)
y_test=np.asarray(y_test)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train/255.0, X_test/255.0
X_validate, y_validate = X_test[:-10], y_test[:-10]
X_test, y_test = X_test[-10:], y_test[-10:]

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam


num_units = 64
num_layers = 1
dropout_rate = 0.2
learning_rate = 0.001
batch_size = 64
num_epochs = 3


model = Sequential()
model.add(LSTM(units=num_units, input_shape=( 28,28), return_sequences=True))
model.add(Dropout(dropout_rate))

for i in range(num_layers-1):
    model.add(LSTM(units=num_units, return_sequences=True))
    model.add(Dropout(dropout_rate))

model.add(LSTM(units=num_units))
model.add(Dropout(dropout_rate))
model.add(Dense(1))

adam = Adam(lr=learning_rate)
model.compile(loss='mse', optimizer='adam', metrics = ["accuracy"])


model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=1, validation_data=(X_validate, y_validate))


loss = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print(f"Test loss: {loss}")


/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/3
938/938 [==============================] - 64s 60ms/step - loss: 3.2619 - accuracy: 0.1567 - val_loss: 1.1992 - val_accuracy: 0.1934
Epoch 2/3
938/938 [==============================] - 48s 52ms/step - loss: 1.1592 - accuracy: 0.1903 - val_loss: 0.6386 - val_accuracy: 0.1979
Epoch 3/3
1/1 [==============================] - 0s 33ms/step - loss: 0.0122 - accuracy: 0.2000
Test loss: [0.012227948755025864, 0.20000000298023224]


In [ ]:
Import confusion_matrix

SyntaxError: ignored

In [ ]:
cm=confusion_matrix(y_test, predictions)

NameError: ignored